In [42]:
import pandas as pd
import numpy as np

# 모델 선택 및 평가 관련 라이브러리
from sklearn.model_selection import train_test_split #,GridSearchCV(하이퍼파라미터), cross_val_score(교차 검증)
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
                            #(accuracy는 맞춘 비율을 계산하여 정확도를 높이지만, 모델이 클래스 구분을 잘 했는지는 파악 불가
                            #roc는 확률값을 활용하여 모델의 성능 평가,, 클래스를 잘 구별하는 지 확인, 1에 가까울수록 좋은 모델
                                #accuracy는 데이터가 균형 잡혀 있을 때 G, roc는 데이터가 불균형할 때 G )

# 전처리: 범주형 인코딩, 결측치 대체, 스케일링
from sklearn.preprocessing import OrdinalEncoder,MinMaxScaler       #범주형 데이터 숫자로 변환
from sklearn.impute import SimpleImputer               #결측치 자동 대체 (strategy='mean':평균값, 'median':중앙값,'most_frequent':최빈값)


In [43]:
# train.csv, test.csv 불러오기 + 불필요 컬럼 삭제(ID)
train = pd.read_csv('./train.csv', encoding='utf-8').drop(columns=['ID'])
test = pd.read_csv('./test.csv', encoding='utf-8').drop(columns=['ID'])

In [44]:
#수민님 전처리 코드

def preprocess_data(df):
    df = df.copy()
    
    #object 타입 -> category 변환 후 Ordinal Encoding
    obj_cols = df.select_dtypes(include=['object']).columns
    if len(obj_cols) > 0:
        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        df[obj_cols] = encoder.fit_transform(df[obj_cols])
    
    #"시술 유형" 유지 + 인코딩
    if "시술 유형" in df.columns:
        df["시술 유형"] = df["시술 유형"].astype(int)  # XGBoost 학습 가능하도록 변환

    #결측값 처리 (시술 유형이 0인 경우 특정 컬럼 결측값을 0으로 대체)
    di_columns = ['해동된 배아 수', '수집된 신선 난자 수', '파트너 정자와 혼합된 난자 수', '이식된 배아 수', 
                  "신선 배아 사용 여부", "동결 배아 사용 여부"]
    
    if "시술 유형" in df.columns:
        df.loc[df['시술 유형'] == 0, di_columns] = df.loc[df['시술 유형'] == 0, di_columns].fillna(0)

    #"시술 당시 나이" 변환 (텍스트 → 숫자)
    age_mapping = {"만18-34세": 0, "만35-37세": 1, "만38-39세": 2, "만40-42세": 3, "만43-44세": 4, "만45-50세": 5}
    df['시술 당시 나이'] = df['시술 당시 나이'].map(age_mapping).fillna(df['시술 당시 나이'].mean())

    #시술 횟수 변환
    count_cols = ["총 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수"]
    mapping = {"0회": 0, "1회": 1, "2회": 2, "3회": 3, "4회": 4, "5회": 5, "6회 이상": 6}
    for col in count_cols:
        df[col] = df[col].map(mapping)

    #정규화 (MinMaxScaler)
    scaler = MinMaxScaler()
    normalize_cols = ["시술 당시 나이", "IVF 임신 횟수", "IVF 출산 횟수", "DI 임신 횟수", "DI 출산 횟수",
                      "이식된 배아 수", "해동된 배아 수", "수집된 신선 난자 수", "파트너 정자와 혼합된 난자 수"]
    df[normalize_cols] = scaler.fit_transform(df[normalize_cols])

    return df


In [45]:
#train 데이터를 train/val로 분할
train.rename(columns={"임신 성공 여부": "target"}, inplace=True)

x = train.drop(columns=["target"])
y = train["target"]

x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [46]:
#전처리
x_train_enc = preprocess_data(x_train)
x_val_enc = preprocess_data(x_val)

c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [47]:
#범주형 & 수치형 변수 정의
categorical_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부",
    "배란 유도 유형", "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부", "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인",
    "여성 부 불임 원인", "부부 주 불임 원인", "부부 부 불임 원인"
]
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수", "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
    "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수"
]

In [48]:
#수치형 변수 결측치 처리 (SimpleImputer 사용)
imputer = SimpleImputer(strategy='median')
X_train_enc[numeric_columns] = imputer.fit_transform(X_train_enc[numeric_columns])
X_val_enc[numeric_columns] = imputer.transform(X_val_enc[numeric_columns])

**XGBoost 모델 학습**

In [49]:
from xgboost import XGBClassifier

# n_estimators를 크게 잡고 early_stopping_rounds를 통해 최적의 에포크에서 중단
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='logloss'
)

In [50]:
#fit 변수 eval_set, early stopping 사용
xgb_model.fit(
    x_train_enc, y_train,
    eval_set=[(x_val_enc, y_val)],
    #early_stopping_rounds=10,
    verbose=True
)

[0]	validation_0-logloss:0.56141
[1]	validation_0-logloss:0.55273
[2]	validation_0-logloss:0.54573
[3]	validation_0-logloss:0.53984
[4]	validation_0-logloss:0.53442
[5]	validation_0-logloss:0.53010
[6]	validation_0-logloss:0.52644
[7]	validation_0-logloss:0.52329
[8]	validation_0-logloss:0.52053
[9]	validation_0-logloss:0.51804
[10]	validation_0-logloss:0.51585
[11]	validation_0-logloss:0.51395
[12]	validation_0-logloss:0.51232
[13]	validation_0-logloss:0.51085
[14]	validation_0-logloss:0.50953
[15]	validation_0-logloss:0.50837
[16]	validation_0-logloss:0.50739
[17]	validation_0-logloss:0.50651
[18]	validation_0-logloss:0.50584
[19]	validation_0-logloss:0.50521
[20]	validation_0-logloss:0.50460
[21]	validation_0-logloss:0.50402
[22]	validation_0-logloss:0.50370
[23]	validation_0-logloss:0.50323
[24]	validation_0-logloss:0.50282
[25]	validation_0-logloss:0.50241
[26]	validation_0-logloss:0.50227
[27]	validation_0-logloss:0.50193
[28]	validation_0-logloss:0.50166
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [51]:
# 검증 데이터 ROC-AUC 평가
val_pred_proba = xgb_model.predict_proba(x_val_enc)[:, 1]
val_roc_auc = roc_auc_score(y_val, val_pred_proba)
print("XGBoost Validation ROC-AUC:", val_roc_auc)

XGBoost Validation ROC-AUC: 0.7206431346225073


In [52]:
#GridSearchCV를 활용한 하이퍼파라미터 튜닝 (마지막에 해보기. 디버깅문제)
"""
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4, 6, 8],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
grid_search = GridSearchCV(
    estimator=XGBClassifier(random_state=42, eval_metric='logloss'),
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)"""

"\nparam_grid = {\n    'n_estimators': [100, 200, 300],\n    'learning_rate': [0.01, 0.1, 0.2],\n    'max_depth': [4, 6, 8],\n    'subsample': [0.8, 1.0],\n    'colsample_bytree': [0.8, 1.0]\n}\ngrid_search = GridSearchCV(\n    estimator=XGBClassifier(random_state=42, eval_metric='logloss'),\n    param_grid=param_grid,\n    scoring='roc_auc',\n    cv=3,\n    n_jobs=-1,\n    verbose=1\n)"

In [53]:
#전체 학습 데이터 전처리 적용
x_full_enc = preprocess_data(X)
x_full_enc[numeric_columns] = imputer.transform(x_full_enc[numeric_columns])

c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [54]:
#최종 XGBoost 모델 (단, early stopping 없이 기본 n_estimators 사용)
model_full = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='logloss'
)
model_full.fit(x_full_enc, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [55]:
#테스트 데이터 전처리 및 예측
x_test_enc = preprocess_data(test)
x_test_enc[numeric_columns] = imputer.transform(x_test_enc[numeric_columns])

test_pred_proba = model_full.predict_proba(x_test_enc)[:, 1]

c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\양소정\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [56]:
# 제출 파일 생성
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = test_pred_proba
sample_submission.to_csv('./0217.02.csv', index=False)
print("Finish! Submission file saved as 0217.02.csv")

Finish! Submission file saved as 0217.02.csv
